<h1>Machine Learning for Sound Classification: Assignment</h1>

### Preliminaries:
* This assignment is worth 10% of the course grade
* The assignment is *individual*. You are welcome to discuss/debate/consult your fellow students, but every student is expected to write their own code, run their own experiments, and answer the assignment questions themselves. The only way to learn is through personal experience!
* The assigment is due 1 week from the date of the class, i.e. by 11:59PM EST on Monday, June 13th, 2016. 
* **There will be no deadline extesions. Assignments will not be accepted over email. Failing to submit the assignment on time will autmatically result in a grade of 0 for the assignment.**
* Each task in this assignment is worth a certain amount of points (specified in the task title), summing to 10 for a perfect score.
* Questions that should be answered with text (rather than code) are indicated with **Q:**
* Have any questions? Found a typo? Please email me at: justin.salamon@nyu.edu

### Description:

Through this assignment you will gain experience working on a real-world dataset (as opposed to a "toy" dataset) that is used for research on urban sound classification. Your goal will be to train and evaluate a multi-class classifier using MFCC features. You may use the ```Sound_classifier``` notebook to guide your work. You may also find it helpful to read through the other example notebooks in this repository before starting work on this assignment.

### How to complete the assignment:

The *entire* assignment must be completed a in single, runnable (i.e. functional) iPython notebook. If you use code from the example notebooks, incorporate it into your assignment notebook and execute it there. Tasks should be completed in the order they appear, and answers to questions should be written in Markdown cells. Please include a title header for each task (and subtask) so that it is clear which task the code/text belongs to. For example, the answer to Task 1 would look like this:

```--------- this is an example, not an actual task ---------```

Task 1
-----
The UrbanSound8K dataset has been downloaded, and stored in the followin path: ```/path/to/data```.

**Q1.1**: There are [your answer here] many sound classes in the dataset, which are: [your anwer here]

**Q1.2**:  A priori, I think the [your answer here] and [your answer here] classes will be harder for the model to distinguish because [your answer here]. I also think that...

**Q1.3**: Based on my name 'justinsalamon', I will be using fold4 for testing, fold5 for validaiton, and the rest of the folds for training.

```--------------------- end of example ---------------------```

All code must be well documented, clearly explaining what you are doing and why (for reference, you should aim to provide the same level of detail as the code comments in the ```Sound_classifier``` notebook.

To submit your assignmet upload your notebook to NYU Classes *before the deadline*. It is *imperative* that your notebook includes the output of each cell (if the cell has any output such as results, print commands or plots). **Notebooks without cell output will not be evaluated and graded 0 automatically**.

------------------- ASSIGNMENT STARTS HERE -------------------
==

Task 1: Getting the data (0.5 point)
--
Download the UrbanSound8K dataset: https://serv.cusp.nyu.edu/projects/urbansounddataset/ and provide the path you'll be using to access the data.
* **Q1.1:** How many different sound classes are there in this dataset and what are they? 
* **Q1.2:** A-priori, which pairs of classes do you think will be the most confused by the model? why?

The dataset comes pre-sorted into 10 folds, named fold1--fold10. We will use the folds as follows:
* 8 folds for training
* 1 fold for validation
* 1 fold for testing

The validation fold can be used to evaluate different features/models/hyper-parameters. This is done by training the model on the training folds (all combined), and testing on the validation fold. Once you have identified the optimal model configuration (features / model / model parameters), you can re-train your model using the training *and* validation folds combined, and finally evaluate the model on the test fold.

You *must* choose which folds to use as follows:
* test fold: ```len('firstnamelastname') % 10 + 1```. For example ```len('justinsalamon') % 10 + 1 = 4```, so I will use fold4 as my test fold.
* validation fold: ```test_fold % 10 + 1```. In our example, my validation fold is fold5.
* training folds: all remaining folds (combined!)

**Q1.3:** describe your personal split: which folds are you using for training, validation, and testing?

Task 2: Preprocessing (1 point)
--

The audio files in UrbanSound8K are all in wav format, but have different sampling rates, bit depths and number of channels. Using the ```Convert_audio_format``` notebook as a guide (or any alternative method of your choice), preprocess all the audio files in the dataset into the following format:
* wav
* sample rate: 44100
* bit depth: 16
* number of channels: 1

Remember that all code used for your assignment must be executed in your assignment notebook.

Task 3: Raw feature extraction (1 points)
--

Using librosa, extract MFCC matrices from all the audio files in the dataset. For the MFCC computation use:
* A frame (window) size of 1024 samples
* A hop size of 512 samples
* 128 mel bands 
* keep the first 25 MFCC coefficients (not including coefficient 0).

**Tip 1**: feature extraction on a moderately large dataset such as this can take a long time. **It is strongly recommended** that you save the raw feaures (mfcc matrices) to disk so that you can easily load them and experiment with different feature summarization / classification models / model parameters without having to recompute the mfcc matrices each time. Assuming your ```matrix``` is a numpy ```ndarray```, an easy way to do this is: ```matrix.dump('filename.npy')```. Loading the data back is as simple as: ```matrix = np.load('filename.npy')```.

**Tip 2**: you can speed up the feature extraction process using multiprocessing:
```
from multiprocessing import Process
...
[Stack overflow will give you the answer]
...
```
For example, extracting the features for each fold in a parallel process would be roughly 10 times faster (if your machine has 10 cores or at least 5 cores with multithreading).

Task 4: Feature processing (1.5 points)
--

Now we need to convert our raw features (mfcc matrices) into feature vectors and stack them into a feature list, and create a correpsonding label list. For each of your three sets (training, validation, test), perform the following:

<h4>Step 1 (1 points):</h4>
For every audio file, summarize each coefficient time-series in the mfcc matrix (i.e. each row) using the following summary statistics: 
* mean
* standard deviation

Combine the sumamry statistics into a flat feature vector and add it to the feature list.

For every file, you also need to figure out the label (class). The mapping from filename to label is provided in the UrbanSound8K.csv file (the "class" column) inside the metadata folder. Once you have identified the class of the file, you need to map it to an integer label from 0 to (nclasses - 1). The recommended way to do this is creating a list of all the class labels and sorting it alphabetically. Then the integer label for each class name is its index (location) in the list (this is how the ```Sound_classifier``` notebook does it).

Once you've obtained the integer label for the audio file, add it to the label list.

**tip**: Use ```pandas``` to load and search through the metadata file.

**tip**: Since there are more than 2 classes in this dataset, the integer labels won't be just 0 and 1 (there will also be 2, 3... etc.)

**tip**: The order in which you populate the feature list and the label list is important, in that they must be populated in the same order. That is, if the first item in the feature list is the feature vector for file X, the first item in the label list must be the integer label for the same file X.

<h4>Step 2 (0.5 point):</h4>
Now that the feature lists (and corresponding label lists) are computed, standardize the feature vectors across dimensions. Remember to learn the standardization parameters only from the training data, and then apply the standardization to all three sets: training, validation, and test.

Task 5: Experimenting on the validation set (3 points)
--

**Step 1 (1 point):**
* Train a K nearest neighbours classifier using the training set (you'll need both the feature list and label list to accomplish this) with K=5 (5 neighbours).
* Test your model on the validation set
* **Q5.1:** What is the classification accuracy?
* **Q5.2:** Display the confusion matrix. Which classes are confused the most?

**Step 2 (1 point):**
* Repeat the train/test procedure (always using the validation set for testing) using K = 3, 7, 13.
* **Q5.3:** How does the accuracy changes as a function of K? Why?

**Step 3 (1 point):**
* Repeat the train/test procedure using each of the following models:
    * Support Vector Machine with a radial basis function kernel.
    * Random Forest with 500 trees

**Q5.4:** How do these models perform compared to the nearest neighbour classifier? Discuss in terms of model accuracy and confusion.

Task 6: Playing with the features (1 points)
--

Repeat steps 1 and 2 of Task 4, modifying them to obtain an extended set of summary statistics:
* mean
* standard deviation
* **mean of the delta mfcc**
* **standard deviation of the delta mfcc**

Train/test the model (using the validation set for testing) using the model that gave you the best performance in Task 5.

**Q6.1:** How do the results change? Discuss in terms of accuracy and confusion. Why?

Task 7: Model evaluation using the test set (2 points)
--

Now that we've finished experimenting with the features and the models and (hopefully) identified the best combination using the validation set, it's time to evaluate the model on the test set.

** Step 1, quantitative analysis (1 point):**
* Using the best feature set identified in previous tasks, combine the training and validation sets into a single large training set
* Use the combined training set to train the best model identified in Task 5.
* Test the model using the **test set**
* **Q7.1**: How did your model perform? Discuss in terms of accuracy and confusion.
* **Q7.2**: Did the model perform better or worse than you expected? Why?

**Step 2: qualitative analysis (1 point):**
* Choose 3 audio files that were miss-classified in Step 1 (each file should correspond to a different sound class)
* **Q7.3**: Listen to each file and suggest a possible explanation for why the model miss-classified it based on how it sounds.

Feedback
--

If you have any (constructive) feedback regarding the lecture and/or the assignment, please include it at the end of your assignment notebook. Thanks!